# M2608.001300 Machine Learning<br> Assignment #5 Final Projects (Pytorch)


## Cap2TxT : An End-to-End Hybrid Neural Network for Captcha Image Text Sequence Recognition

<div style="text-align: right">
Changwoon Choi.<br>
SNU ECE.<br>
2014-17733<br>
</div>

For code simplication of this main.ipynb file, I added some python files for simple calculation or transformation at current directory.<br> (those codes are also submitted with this main.ipynb file with maintaining the directory hierarchy.)

In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import CTCLoss
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt

import dataset
import utils
import params

Load datasets


In [6]:
alphabet='0123456789abcdefghijklmnopqrstuvwxyz'
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NONE = ['NONE'] # label for empty space
ALL_CHAR_SET = NUMBER + ALPHABET + NONE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 7

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

class Mydataset(Dataset):
    def __init__(self, img_path, label_path, is_train=True, transform=None):
        self.path = img_path
        self.label_path = label_path
        if is_train: 
            self.img = os.listdir(self.path)[:10000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:10000]
        else: 
            self.img = os.listdir(self.path)[:1000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:1000]
        
        self.transform = transform
        self.max_length = MAX_CAPTCHA
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(f'{self.path}/{self.img[idx]}')
        img = img.convert('L')
        label = self.labels[idx]
        label_oh = []
        
        for i in range(self.max_length):
            if i < len(label):
                label_oh += encode(label[i])
            else:
                #label_oh += [0]*ALL_CHAR_SET_LEN
                label_oh += encode('NONE')
            
        if self.transform is not None:
            img = self.transform(img)

        return img, np.array(label_oh), label
    def __len__(self):
        return len(self.img)

transform = transforms.Compose([
    transforms.Resize([160, 60]),
    transforms.ToTensor()
])

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)


In [7]:
gPath = './'
train_ds = Mydataset(gPath+'Data/train/', gPath+'Data/train.txt',transform=transform)
test_ds = Mydataset(gPath+'Data/test/', gPath+'Data/test.txt',False, transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4)
my_train_loader=torch.utils.data.DataLoader(dataset.lmdbDataset('Data/train_prepared'), batch_size=32,shuffle=True, sampler=None, num_workers=4, collate_fn=dataset.alignCollate(imgH=64,imgW=160, keep_ratio=False))

In [ ]:
#prepare dataset (the python files below should be executed only once.)

In [14]:
!python misc.py

In [15]:
!python misc_2.py

In [17]:
!python misc_3.py

In [10]:
!python create_dataset.py --out Data/train_prepared --folder Data/train

create_dataset.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  imageBuf = np.fromstring(imageBin, dtype=np.uint8)


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Problem 1: Design LSTM model for captcha image recognition. (10 points)

In [42]:
class ONE_LAYER_LSTM(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(ONE_LAYER_LSTM,self).__init__()
        self.lstm = nn.LSTM(dim_in, dim_hidden, bidirectional=True)
        self.fc_out = nn.Linear(2*dim_hidden, dim_out)
    def forward(self, features):
        tmp, _ = self.lstm(features)
        T,b,h = tmp.size()
        tmp = tmp.view(T*b,h)
        output = self.fc_out(tmp)
        output = output.view(T,b,-1)
        return output

class LSTM(nn.Module):
    def __init__(self, cnn_dim, hidden_size, classnum, num_layers=2):
        super(LSTM, self).__init__()
        self.lstmnet = nn.Sequential(ONE_LAYER_LSTM(cnn_dim, hidden_size, hidden_size),
                                    ONE_LAYER_LSTM(hidden_size,hidden_size, classnum))
        
    def forward(self, features):
        output = self.lstmnet(features)
        output = F.log_softmax(output,dim=2)
        return output




Problem 2: 

*   1.Connect CNN model to the designed LSTM model.
*   2.Replace ResNet to your own CNN model from Assignment3.


          


In [54]:
# trial1
'''my betternet from AS3'''

class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )
    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)
    
class BetterNet_1(nn.Module):
    def __init__(self):
        super(BetterNet_1, self).__init__()        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=1,padding=3),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=1,stride=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1,padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,1)
        )
        self.incep_block1 = nn.Sequential(
            Inception(192,64,96,128,16,32,32),
            Inception(256,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1)
        )        
        self.incep_block2 = nn.Sequential(
            Inception(480,192,96,208,16,48,64),
            Inception(512,160,112,224,24,64,64),
            Inception(512,128,128,256,24,64,64),
            Inception(512,112,144,288,32,64,64),
            Inception(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1)
        )        
        self.incep_block3 = nn.Sequential(
            Inception(832,256,160,320,32,128,128),
            Inception(832,384,192,384,48,128,128),
            nn.AvgPool2d(kernel_size=4,stride=1,padding=0)
        )        
        self.fc_block = nn.Sequential(
            nn.Linear(1024,512),
            nn.ReLU(True),
            nn.Linear(512,ALL_CHAR_SET_LEN*MAX_CAPTCHA)
           # nn.ReLU(True),
           # nn.Linear(64,10)
        )        
    def forward(self, x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.incep_block1(x)
        x=self.incep_block2(x)
        x=self.incep_block3(x)
        #print('cnn 5:',end='')
        #print(x.shape)
        #x = x.view(-1,1024)
        #x=self.fc_block(x)
        out=x
        return out


In [46]:
# trial 2
'''my betternet from AS3'''
class BetterNet_2(nn.Module):
    def __init__(self):
        super(BetterNet_2, self).__init__()
        
        self.cnn1=nn.Sequential()
        self.cnn1.add_module('conv1', nn.Conv2d(1,64,3,1,1))
        self.cnn1.add_module('relu1', nn.ReLU(True))
        self.cnn1.add_module('pooling1', nn.MaxPool2d(2,2))
        
        self.cnn2=nn.Sequential()
        self.cnn2.add_module('conv2', nn.Conv2d(64,128,3,1,1))
        self.cnn2.add_module('relu2', nn.ReLU(True))
        self.cnn2.add_module('pooling2', nn.MaxPool2d(2,2))
        
        self.cnn3=nn.Sequential()
        self.cnn3.add_module('conv3', nn.Conv2d(128,256,3,1,1))
        self.cnn3.add_module('batchnorm', nn.BatchNorm2d(256))
        self.cnn3.add_module('relu3', nn.ReLU(True))
               
        self.cnn4=nn.Sequential()
        self.cnn4.add_module('conv4', nn.Conv2d(256,256,3,1,1))
        self.cnn4.add_module('relu4', nn.ReLU(True))
        self.cnn4.add_module('pooling4', nn.MaxPool2d((2,2),(2,1),(0,1)))
        
        self.cnn5=nn.Sequential()
        self.cnn5.add_module('conv5', nn.Conv2d(256,512,3,1,1))
        self.cnn5.add_module('batchnorm', nn.BatchNorm2d(512))
        self.cnn5.add_module('relu5', nn.ReLU(True))

        self.cnn6=nn.Sequential()
        self.cnn6.add_module('conv6', nn.Conv2d(512,512,3,1,1))
        self.cnn6.add_module('relu6', nn.ReLU(True))
        self.cnn6.add_module('pooling6', nn.MaxPool2d((2,2),(2,1),(0,1)))
        
        self.cnn7=nn.Sequential()
        self.cnn7.add_module('conv7', nn.Conv2d(512,512,2,1,0))
        self.cnn7.add_module('batchnorm', nn.BatchNorm2d(512))
        self.cnn7.add_module('relu7', nn.ReLU(True))
        self.cnn7.add_module('pooling7', nn.AvgPool2d(kernel_size=3,stride=1))
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.cnn2(out)
        out=self.cnn3(out)
        out=self.cnn4(out)
        out=self.cnn5(out)
        out=self.cnn6(out)
        out=self.cnn7(out)
        return out
        
        


In [12]:
def weight_normalize(net):
    a=net.__class__.__name__
    if a.find('Conv') != -1:
        net.weight.data.normal_(0,0.02)
    elif a.find('BatchNorm') !=-1:
        net.weight.data.normal_(1,0.02)
        net.bias.data.fill_(0)

In [55]:
# my CNN-RNN fusioned model, Cap2TxT Network
# trial 1,2
vocab_size = len(alphabet)+1

class Cap2TxT(nn.Module):
    def __init__(self, hidden_size, class_num, num_layers=2):
        super(Cap2TxT, self).__init__()
        #self.cnn = BetterNet_2()
        self.cnn = BetterNet_1()
        #self.rnn = LSTM(512, hidden_size, class_num,num_layers)
        self.rnn = LSTM(1024, hidden_size, class_num, num_layers)
    def forward(self, x):
        
        features = self.cnn(x)
        print(features.shape)
        b,c,h,w = features.size()
        features = features.squeeze(2)
        features = features.permute(2,0,1)
        output = self.rnn(features)
        return output


hidden_size=256
batch_size = 32
lr = 0.0001

image = torch.FloatTensor(batch_size, 3, 64, 64)
text = torch.LongTensor(batch_size * 5)
length = torch.LongTensor(batch_size)

net = Cap2TxT(hidden_size,vocab_size,2)
net.apply(weight_normalize)


converter = utils.strLabelConverter(alphabet)
loss_func = CTCLoss()
loss_avg=utils.averager()

loss_func = loss_func.cuda()
image=image.cuda()
text=text.cuda()
net=net.cuda()

image=Variable(image)
text=Variable(text)
length=Variable(length)

cap2txt_optim = optim.RMSprop(net.parameters(),lr)
#cap2txt_optim = optim.Adam(net.parameters(),lr,betas=(0.5,0.999))



Problem3: Find hyper-parameters.


In [42]:
"""TRAINING"""

SAVE_PATH = './ckpt/'
display_interval = 1
save_interval=3
max_epoch = 250


for epoch in range(max_epoch):
    train_it = iter(my_train_loader)
    it=0
    while it<len(my_train_loader):
        for param in net.parameters():
            param.requires_grad=True
        net.train()
        chunk = train_it.next()
        img, label = chunk
        utils.loadData(image,img)
        txt, lth = converter.encode(label)
        utils.loadData(text,txt)
        utils.loadData(length,lth)
        
        cap2txt_optim.zero_grad()
        predict = net(image)
        predict_size = Variable(torch.LongTensor([predict.size(0)] * img.size(0)))
        
        loss = loss_func(predict,text,predict_size,length)/img.size(0)
        loss.backward()
        cap2txt_optim.step()
        
        loss_avg.add(loss)
        it+=1
        
    if epoch%display_interval==0:
        print('epoch : %d, Loss : %f' %(epoch,loss_avg.val()))
        loss_avg.reset()
    if epoch%save_interval==0:
        print('saved')
        #torch.save(net.state_dict(),SAVE_PATH="Cap2TxT_"+str(epoch)+'pth')
        torch.save(net.state_dict(),SAVE_PATH+'Cap2TxT_GoogLe_'+str(epoch)+'.pth')

print('Finished Training')
torch.save(net.state_dict(),SAVE_PATH+'Cap2TxT'+'_final.pth')
print('Saved Model')

epoch : 0, Loss : 0.122119
saved
epoch : 1, Loss : 0.114869
epoch : 2, Loss : 0.097594
epoch : 3, Loss : 0.080764
saved
epoch : 4, Loss : 0.063179
epoch : 5, Loss : 0.047502
epoch : 6, Loss : 0.036180
saved
epoch : 7, Loss : 0.029120
epoch : 8, Loss : 0.022502
epoch : 9, Loss : 0.018161
saved
epoch : 10, Loss : 0.015251
epoch : 11, Loss : 0.012913
epoch : 12, Loss : 0.010618
saved
epoch : 13, Loss : 0.009241
epoch : 14, Loss : 0.007873
epoch : 15, Loss : 0.007073
saved
epoch : 16, Loss : 0.006063
epoch : 17, Loss : 0.005284
epoch : 18, Loss : 0.005042
saved
epoch : 19, Loss : 0.006001
epoch : 20, Loss : 0.003667
epoch : 21, Loss : 0.004812
saved
epoch : 22, Loss : 0.004109
epoch : 23, Loss : 0.003277
epoch : 24, Loss : 0.002493
saved
epoch : 25, Loss : 0.002988
epoch : 26, Loss : 0.002113
epoch : 27, Loss : 0.002185
saved
epoch : 28, Loss : 0.002145
epoch : 29, Loss : 0.001959
epoch : 30, Loss : 0.001460
saved
epoch : 31, Loss : 0.001873
epoch : 32, Loss : 0.001267
epoch : 33, Loss : 0

In [61]:
"""TRAINING"""

SAVE_PATH = './ckpt/'
display_interval = 1
save_interval=3
max_epoch = 1


for epoch in range(max_epoch):
    train_it = iter(my_train_loader)
    it=0
    while it<len(my_train_loader):
        for param in net.parameters():
            param.requires_grad=True
        net.train()
        chunk = train_it.next()
        img, label = chunk
        utils.loadData(image,img)
        #print(image)
        txt, lth = converter.encode(label)
        utils.loadData(text,txt)
        #print(text)
        utils.loadData(length,lth)
        #print(length)
        cap2txt_optim.zero_grad()
        predict = net(image)
        predict_size = Variable(torch.LongTensor([predict.size(0)] * img.size(0)))
        print('here')
        print(predict[:,0,:])
        print(text[0,:])
        print(predict_size)
        print(length)
        loss = loss_func(predict,text,predict_size,length)/img.size(0)
        loss.backward()
        cap2txt_optim.step()
        
        loss_avg.add(loss)
        it+=1
        
    if epoch%display_interval==0:
        print('epoch : %d, Loss : %f' %(epoch,loss_avg.val()))
        loss_avg.reset()
    if epoch%save_interval==0:
        print('saved')
        #torch.save(net.state_dict(),SAVE_PATH="Cap2TxT_"+str(epoch)+'pth')
        t#orch.save(net.state_dict(),SAVE_PATH+'Cap2TxT_GoogLe_'+str(epoch)+'.pth')

print('Finished Training')
#torch.save(net.state_dict(),SAVE_PATH+'Cap2TxT'+'_final.pth')
print('Saved Model')

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.9577e+00, -3.5050e+00, -3.7317e+00, -3.5668e+00, -3.5474e+00,
         -3.4850e+00, -3.5377e+00, -3.6249e+00, -3.6213e+00, -3.6172e+00,
         -3.5492e+00, -3.6494e+00, -3.5739e+00, -3.6187e+00, -3.5649e+00,
         -3.5134e+00, -3.6227e+00, -3.6083e+00, -3.6113e+00, -3.7008e+00,
         -3.6817e+00, -3.4374e+00, -3.6909e+00, -3.8928e+00, -3.4895e+00,
         -3.7098e+00, -3.6091e+00, -3.6494e+00, -3.6046e+00, -3.5331e+00,
         -3.6777e+00, -3.4763e+00, -3.5117e+00, -3.8493e+00, -3.5324e+00,
         -3.5408e+00, -3.7342e+00],
        [-3.6584e+00, -3.4096e+00, -3.8467e+00, -3.6683e+00, -3.5553e+00,
         -3.3649e+00, -3.5084e+00, -3.6627e+00, -3.7511e+00, -3.6241e+00,
         -3.5335e+00, -3.6264e+00, -3.5230e+00, -3.7498e+00, -3.5111e+00,
         -3.4376e+00, -3.6989e+00, -3.5725e+00, -3.5086e+00, -3.7872e+00,
         -3.8275e+00, -3.4195e+00, -3.7938e+00, -3.8582e+00, -3.4235e+00,
         -3.7077e+00, -3.5876e+00, -3.6091

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.7243, -3.9630, -3.3583, -3.1902, -3.3420, -3.8593, -3.4688, -3.8180,
         -3.2381, -4.0558, -3.8986, -4.0768, -3.7099, -3.2723, -3.6973, -3.9278,
         -3.2108, -3.8913, -3.7609, -3.3825, -3.1905, -3.4474, -3.3607, -4.7275,
         -3.7695, -3.9088, -4.1175, -4.0744, -3.3390, -3.3747, -3.2048, -3.7586,
         -3.8399, -4.6208, -3.4401, -3.4788, -3.4448],
        [-3.3735, -3.9734, -3.4371, -3.1856, -3.3212, -3.8456, -3.4100, -3.8485,
         -3.2625, -4.1201, -3.8964, -4.0883, -3.6821, -3.3169, -3.6561, -3.8765,
         -3.2380, -3.8772, -3.7043, -3.4192, -3.2185, -3.4353, -3.3953, -4.8837,
         -3.7690, -3.9729, -4.1982, -4.0855, -3.3750, -3.3083, -3.1762, -3.8020,
         -3.8791, -4.7445, -3.5068, -3.4658, -3.4384],
        [-2.1103, -3.8161, -3.7248, -3.4823, -3.4629, -3.6910, -3.4255, -3.8760,
         -3.5252, -4.1047, -3.8315, -4.0106, -3.6033, -3.6385, -3.6250, -3.7779,
         -3.4929, -3.8276, -3.5888, -3.6255, -

torch.Size([32, 1024, 1, 7])
here
tensor([[-4.1804, -3.7671, -3.5010, -3.3427, -3.4473, -3.7484, -3.5146, -3.6856,
         -3.3412, -3.7912, -3.7390, -3.8320, -3.6395, -3.3504, -3.6410, -3.7246,
         -3.3777, -3.7176, -3.7650, -3.5170, -3.3546, -3.4245, -3.4954, -4.3160,
         -3.6608, -3.7626, -3.8458, -3.8583, -3.4511, -3.4320, -3.4244, -3.6313,
         -3.6423, -4.2229, -3.3725, -3.5149, -3.5436],
        [-4.1013, -3.7087, -3.5990, -3.3821, -3.4329, -3.6912, -3.4680, -3.6851,
         -3.4103, -3.7811, -3.6970, -3.7796, -3.5902, -3.4222, -3.5636, -3.6419,
         -3.4465, -3.6403, -3.7057, -3.5895, -3.4477, -3.3869, -3.5779, -4.3389,
         -3.6142, -3.7639, -3.8230, -3.8160, -3.5146, -3.3871, -3.4879, -3.5847,
         -3.6059, -4.2288, -3.3978, -3.5085, -3.5774],
        [-3.5863, -3.4984, -3.8107, -3.5797, -3.5021, -3.4794, -3.4536, -3.6853,
         -3.6412, -3.6744, -3.6107, -3.6545, -3.4630, -3.6617, -3.4704, -3.4701,
         -3.6592, -3.4925, -3.5665, -3.7774, -

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.8395, -3.9088, -3.3477, -3.2539, -3.3789, -3.8626, -3.4766, -3.7781,
         -3.2222, -3.9727, -3.8837, -4.0056, -3.6658, -3.2769, -3.6857, -3.9337,
         -3.2352, -3.8955, -3.7722, -3.3968, -3.2085, -3.3764, -3.4437, -4.7017,
         -3.7360, -3.8404, -3.9701, -4.0407, -3.3672, -3.3953, -3.2705, -3.7368,
         -3.8009, -4.5766, -3.3801, -3.5142, -3.4915],
        [-3.6336, -3.9223, -3.4012, -3.2526, -3.3333, -3.8791, -3.4064, -3.8143,
         -3.2239, -4.0160, -3.8997, -4.0123, -3.6280, -3.2971, -3.6440, -3.9518,
         -3.2385, -3.9110, -3.7479, -3.4216, -3.2118, -3.3304, -3.4851, -4.9149,
         -3.7401, -3.8669, -3.9944, -4.0753, -3.3808, -3.3655, -3.2571, -3.7705,
         -3.8260, -4.7310, -3.3740, -3.4951, -3.4998],
        [-2.7946, -3.9256, -3.5015, -3.3423, -3.3487, -3.8469, -3.3213, -3.8694,
         -3.2848, -4.0457, -3.9269, -4.0393, -3.5664, -3.3718, -3.6205, -3.9958,
         -3.3133, -3.9455, -3.7066, -3.4703, -

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.7560e+00, -3.2524e+00, -4.0739e+00, -3.9809e+00, -3.7741e+00,
         -3.2921e+00, -3.7506e+00, -3.6826e+00, -4.0382e+00, -3.3870e+00,
         -3.4219e+00, -3.3739e+00, -3.5405e+00, -3.9184e+00, -3.6260e+00,
         -3.2996e+00, -3.9712e+00, -3.4433e+00, -3.5662e+00, -4.0133e+00,
         -4.0574e+00, -3.4874e+00, -4.1071e+00, -3.1051e+00, -3.4312e+00,
         -3.6201e+00, -3.4067e+00, -3.3295e+00, -3.9258e+00, -3.8212e+00,
         -4.0948e+00, -3.4540e+00, -3.3725e+00, -3.2675e+00, -3.7245e+00,
         -3.6869e+00, -4.0823e+00],
        [-3.7597e+00, -3.3447e+00, -3.9152e+00, -3.8669e+00, -3.6207e+00,
         -3.3253e+00, -3.5733e+00, -3.6693e+00, -3.8827e+00, -3.5360e+00,
         -3.4894e+00, -3.4393e+00, -3.4535e+00, -3.7628e+00, -3.5861e+00,
         -3.3993e+00, -3.7560e+00, -3.5679e+00, -3.4913e+00, -3.8981e+00,
         -3.8958e+00, -3.3279e+00, -4.0189e+00, -3.6257e+00, -3.4323e+00,
         -3.6259e+00, -3.4895e+00, -3.4272

...

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.9093e+00, -3.7818e+00, -3.4482e+00, -3.3502e+00, -3.3709e+00,
         -3.8106e+00, -3.4862e+00, -3.6792e+00, -3.3315e+00, -3.8713e+00,
         -3.6960e+00, -3.8486e+00, -3.5878e+00, -3.4737e+00, -3.6954e+00,
         -3.7784e+00, -3.2696e+00, -3.8349e+00, -3.6540e+00, -3.4788e+00,
         -3.4155e+00, -3.3819e+00, -3.4253e+00, -4.6380e+00, -3.7072e+00,
         -3.7936e+00, -3.8944e+00, -3.9368e+00, -3.3730e+00, -3.4289e+00,
         -3.4034e+00, -3.6883e+00, -3.6815e+00, -4.5184e+00, -3.3328e+00,
         -3.5070e+00, -3.5139e+00],
        [-3.7028e+00, -3.6635e+00, -3.6079e+00, -3.4857e+00, -3.3783e+00,
         -3.7266e+00, -3.4418e+00, -3.6571e+00, -3.4781e+00, -3.7970e+00,
         -3.6029e+00, -3.7066e+00, -3.4770e+00, -3.6100e+00, -3.6112e+00,
         -3.6406e+00, -3.3724e+00, -3.7580e+00, -3.4988e+00, -3.5921e+00,
         -3.6003e+00, -3.2909e+00, -3.5589e+00, -4.6515e+00, -3.6379e+00,
         -3.7485e+00, -3.7996e+00, -3.8189

torch.Size([32, 1024, 1, 7])
here
tensor([[-4.0994, -3.4269, -3.7470, -3.6593, -3.6224, -3.5381, -3.5913, -3.6126,
         -3.6967, -3.5242, -3.4823, -3.5231, -3.4778, -3.6743, -3.5552, -3.4892,
         -3.6419, -3.5827, -3.5449, -3.7714, -3.7032, -3.4679, -3.7037, -3.9326,
         -3.5126, -3.6279, -3.5124, -3.5872, -3.6606, -3.5939, -3.8181, -3.4556,
         -3.5000, -3.8532, -3.4584, -3.5358, -3.7816],
        [-4.0721, -3.4224, -3.7888, -3.6917, -3.5790, -3.5336, -3.5560, -3.6165,
         -3.7289, -3.5398, -3.4754, -3.4829, -3.4434, -3.7006, -3.5174, -3.4773,
         -3.6299, -3.5874, -3.4748, -3.8044, -3.7505, -3.4056, -3.7545, -4.1143,
         -3.4893, -3.6077, -3.5118, -3.5856, -3.6927, -3.5624, -3.8724, -3.4402,
         -3.4863, -3.9818, -3.4360, -3.5201, -3.8152],
        [-3.6796, -3.3981, -3.8804, -3.7768, -3.5978, -3.4956, -3.5523, -3.6134,
         -3.8059, -3.5263, -3.4742, -3.4250, -3.3966, -3.7804, -3.4921, -3.4175,
         -3.7016, -3.5819, -3.3780, -3.8639, -

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.7257, -3.4502, -3.7519, -3.7398, -3.6151, -3.5127, -3.5516, -3.6077,
         -3.6681, -3.5311, -3.5315, -3.5490, -3.5170, -3.6651, -3.6338, -3.5419,
         -3.5815, -3.6788, -3.5303, -3.7322, -3.6978, -3.4142, -3.8125, -3.8752,
         -3.5086, -3.5996, -3.4430, -3.6191, -3.6553, -3.6497, -3.7952, -3.4957,
         -3.5125, -3.8164, -3.5044, -3.5380, -3.7977],
        [-3.5270, -3.7541, -3.4292, -3.5426, -3.3565, -3.7722, -3.3973, -3.6823,
         -3.3774, -3.8387, -3.7907, -3.7943, -3.6485, -3.4055, -3.6716, -3.9092,
         -3.2100, -3.9099, -3.5761, -3.5116, -3.3736, -3.3806, -3.5944, -4.6924,
         -3.6867, -3.7197, -3.7399, -3.9885, -3.3972, -3.5623, -3.4889, -3.6529,
         -3.7033, -4.5116, -3.3546, -3.4154, -3.5955],
        [-2.6107, -4.3198, -3.1313, -3.3122, -3.1806, -4.3022, -3.3769, -3.9268,
         -3.0852, -4.3973, -4.2519, -4.2656, -4.0335, -3.2092, -3.8891, -4.4396,
         -2.9275, -4.2804, -3.8215, -3.3086, -

torch.Size([32, 1024, 1, 7])
here
tensor([[-3.8432e+00, -3.3922e+00, -3.8004e+00, -3.7625e+00, -3.6489e+00,
         -3.5113e+00, -3.5585e+00, -3.5742e+00, -3.7327e+00, -3.5331e+00,
         -3.3778e+00, -3.4944e+00, -3.5094e+00, -3.7799e+00, -3.4665e+00,
         -3.4523e+00, -3.6308e+00, -3.5653e+00, -3.5028e+00, -3.7431e+00,
         -3.7819e+00, -3.3465e+00, -3.7719e+00, -4.0832e+00, -3.4710e+00,
         -3.7052e+00, -3.4768e+00, -3.6335e+00, -3.7248e+00, -3.6295e+00,
         -3.9019e+00, -3.4430e+00, -3.4484e+00, -4.0282e+00, -3.4854e+00,
         -3.4680e+00, -3.9116e+00],
        [-3.5121e+00, -3.2548e+00, -4.0242e+00, -3.9658e+00, -3.7433e+00,
         -3.4205e+00, -3.5852e+00, -3.6058e+00, -3.9453e+00, -3.4528e+00,
         -3.3218e+00, -3.3729e+00, -3.4234e+00, -3.9720e+00, -3.3956e+00,
         -3.3553e+00, -3.7791e+00, -3.5189e+00, -3.3883e+00, -3.8481e+00,
         -4.0332e+00, -3.3068e+00, -3.9733e+00, -4.0488e+00, -3.3680e+00,
         -3.6839e+00, -3.3509e+00, -3.5482

KeyboardInterrupt: 

In [37]:
'''Test'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh



#model_path = '../final/ckpt/netCRNN_80_266.pth'

#best at Cap2TxT
model_path ='./ckpt/Cap2TxT_180.pth'

#best at Cap2TxT_GoogLeNet


image_path = './Data/test/'

pred_list =[]

class_num = len(alphabet) + 1

model = Cap2TxT(hidden_size,class_num)
#model = Cap2TxT_2(64,1,class_num,hidden_size)

model = model.cuda()

model.load_state_dict(torch.load(model_path))
model.eval()
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))
for i in range(1000):
    image = Image.open(image_path+str(i)+'.png').convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.LongTensor([preds.size(0)]))
    pred = converter.decode(preds.data, preds_size.data, raw=False)
    pred_list.append(pred)

answer='./Data/test.txt'
answer_list=list()
with open (answer) as f:
    for line in f:
        answer_list.append(line.rstrip('\n'))

char_correct = 0
word_correct = 0
total = 0

for i in range(1000): # size of test set is 1000
    char_count=0
    c0,c1,c2,c3,c4,c5,c6 = get_char_count(oh_encoding(pred_list[i]))
    d0,d1,d2,d3,d4,d5,d6 = get_char_count(oh_encoding(answer_list[i]))
    c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
    d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
    char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
    char_correct += char_count
    if(bool(str(answer_list[i]) in str(c))):
        word_correct+=1
    total += 1

print('---char correct---')
print(100*(char_correct/(total*7)), end=' ')
print('%')
print('---word correct---')
print(100*word_correct/total, end=' ')
print('%')



---char correct---
77.55714285714286 %
---word correct---
62.6 %


In [58]:
'''Find the best weight from my experiment results.'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh



#model_path = '../final/ckpt/netCRNN_80_266.pth'

#best at Cap2TxT
#model_path ='./ckpt/Cap2TxT_180.pth'

#best at Cap2TxT_GoogLeNet
#model_path = './ckpt/Cap2TxT_final.pth'
#model_path = './ckpt/Cap2TxT_GoogLe_150.pth'
image_path = './Data/test/'
class_num = len(alphabet) + 1
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))

answer='./Data/test.txt'
answer_list=list()
with open (answer) as f:
    for line in f:
        answer_list.append(line.rstrip('\n'))

#Let's find best fitted model among 0~250 epoch that performs best at test set!

max_accuracy = 0
best_epoch = 0
epoch=0
while epoch <250 :
    model_path = './ckpt/Cap2TxT_GoogLe_'+str(epoch)+'.pth'
    pred_list =[]
    model = Cap2TxT(hidden_size,class_num)
    #model = Cap2TxT_2(64,1,class_num,hidden_size)
    model = model.cuda()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    for i in range(1000):
        image = Image.open(image_path+str(i)+'.png').convert('L')
        image = transformer(image)
        if torch.cuda.is_available():
            image = image.cuda()
        image = image.view(1, *image.size())
        image = Variable(image)
        preds = model(image)

        _, preds = preds.max(2)
        preds = preds.transpose(1, 0).contiguous().view(-1)
        preds_size = Variable(torch.LongTensor([preds.size(0)]))
        pred = converter.decode(preds.data, preds_size.data, raw=False)
        pred_list.append(pred)

    char_correct = 0
    word_correct = 0
    total = 0

    for i in range(1000): # size of test set is 1000
        char_count=0
        c0,c1,c2,c3,c4,c5,c6 = get_char_count(oh_encoding(pred_list[i]))
        d0,d1,d2,d3,d4,d5,d6 = get_char_count(oh_encoding(answer_list[i]))
        c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
        d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
        char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
        char_correct += char_count
        if(bool(str(answer_list[i]) in str(c))):
            word_correct+=1
        total += 1
    if max_accuracy < (100*(char_correct/(total*7))+100*word_correct/total):
        max_accuracy = (100*(char_correct/(total*7))+100*word_correct/total)
        best_epoch = epoch
    print("------------------------------------")
    print("current...")
    print(epoch, end=':')
    print(100*(char_correct/(total*7))+100*word_correct/total)
    print("best...")
    print(best_epoch, end=":")
    print(max_accuracy)
    epoch += 3



------------------------------------
current...
0:36.72857142857143
best...
0:36.72857142857143
------------------------------------
current...
3:38.2
best...
3:38.2
------------------------------------
current...
6:36.32857142857143
best...
3:38.2
------------------------------------
current...
9:101.45714285714286
best...
9:101.45714285714286
------------------------------------
current...
12:111.01428571428572
best...
12:111.01428571428572
------------------------------------
current...
15:121.2
best...
15:121.2
------------------------------------
current...
18:86.57142857142857
best...
15:121.2
------------------------------------
current...
21:147.22857142857143
best...
21:147.22857142857143
------------------------------------
current...
24:148.51428571428573
best...
24:148.51428571428573
------------------------------------
current...
27:138.38571428571427
best...
24:148.51428571428573
------------------------------------
current...
30:49.800000000000004
best...
24:148.51428571

In [15]:
best_epoch=231
print(best_epoch)

231


In [16]:
'''Test'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh

#best weight of Cap2TxT Network.
model_path ='./ckpt/Cap2TxT_GoogLe_'+str(best_epoch)+'.pth'
image_path = './Data/test/'

pred_list =[]

class_num = len(alphabet) + 1

model = Cap2TxT(hidden_size,class_num)
model = model.cuda()

model.load_state_dict(torch.load(model_path))
model.eval()
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))
for i in range(1000):
    image = Image.open(image_path+str(i)+'.png').convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    preds = model(image)

    _, preds = preds.max(2)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.LongTensor([preds.size(0)]))
    pred = converter.decode(preds.data, preds_size.data, raw=False)
    pred_list.append(pred)

answer='./Data/test.txt'
answer_list=list()
with open (answer) as f:
    for line in f:
        answer_list.append(line.rstrip('\n'))

char_correct = 0
word_correct = 0
total = 0

for i in range(1000): # size of test set is 1000
    char_count=0
    c0,c1,c2,c3,c4,c5,c6 = get_char_count(oh_encoding(pred_list[i]))
    d0,d1,d2,d3,d4,d5,d6 = get_char_count(oh_encoding(answer_list[i]))
    c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
    d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
    char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
    char_correct += char_count
    if(bool(str(answer_list[i]) in str(c))):
        word_correct+=1
    total += 1

print('---char correct---')
print(100*(char_correct/(total*7)), end=' ')
print('%')
print('---word correct---')
print(100*word_correct/total, end=' ')
print('%')

---char correct---
88.2 %
---word correct---
77.9 %


In [51]:
'''debug'''

def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 

def oh_encoding(a):
    label_oh = []
    for i in range(7):
        if i<len(a):
            label_oh+=encode(a[i])
        else:
            label_oh+=encode('NONE')
    return label_oh

#best weight of Cap2TxT Network.
model_path ='./ckpt/Cap2TxT_GoogLe_'+str(best_epoch)+'.pth'
image_path = './Data/test/'

pred_list =[]

class_num = len(alphabet) + 1

model = Cap2TxT(hidden_size,class_num)
model = model.cuda()

model.load_state_dict(torch.load(model_path))
model.eval()
converter = utils.strLabelConverter(alphabet)
transformer=dataset.resizeNormalize((160,64))
for i in range(1):
    image = Image.open(image_path+str(i)+'.png').convert('L')
    image = transformer(image)
    if torch.cuda.is_available():
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    print(image.shape)
    preds = model(image)
    print(preds.shape)
    _, preds = preds.max(2)
    print(pred)
    preds = preds.transpose(1, 0).contiguous().view(-1)
    preds_size = Variable(torch.LongTensor([preds.size(0)]))
    pred = converter.decode(preds.data, preds_size.data, raw=False)
    print(pred)
    pred_list.append(pred)

torch.Size([1, 1, 64, 160])
cnn out
torch.Size([1, 1024, 1, 7])
torch.Size([1, 1024, 1, 7])
torch.Size([7, 1, 1024])
torch.Size([7, 1, 37])
torch.Size([7, 1, 37])
b9x
b9x
